#### Loading the Dataset for Text classification

In [ ]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz


# The dataset is about 80mb

In [ ]:
!rm -r aclImdb/train/unsup

In [ ]:
import os, pathlib, shutil, random
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"

for category in ("neg", "pos"):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = int(0.2 * len(files))
    val_files = files[-num_val_samples:]

for fname in val_files:
  shutil.move(train_dir / category / fname,
              val_dir / category / fname)

In [ ]:
from tensorflow import keras
batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(
"aclImdb/train", batch_size=batch_size)

val_ds = keras.utils.text_dataset_from_directory(
 "aclImdb/val", batch_size=batch_size)

test_ds = keras.utils.text_dataset_from_directory(
 "aclImdb/test", batch_size=batch_size)

#### Text Classification Using LSTM

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
text_only_train_ds = train_ds.map(lambda x, y: x)
# give only the raw text input no result

In [ ]:
max_length = 600
max_tokens = 20000
text_vectorization = layers.TextVectorization(
  max_tokens=max_tokens,
  output_mode="int",
  output_sequence_length=max_length,
)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y),
  num_parallel_calls=4)

int_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y),
  num_parallel_calls=4)

int_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y),
  num_parallel_calls=4)

In [ ]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
    loss="binary_crossentropy",
    metrics=["accuracy"])
model.summary()

In [ ]:
callbacks = [
  keras.callbacks.ModelCheckpoint("embeddings_bidir_gru.keras",
  save_best_only=True)
]

model.fit(int_train_ds, validation_data=int_val_ds, epochs=10,
callbacks=callbacks)

model = keras.models.load_model("embeddings_bidir_gru.keras")

print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

#### Transformer Architecture for Text Classfication

Encoder only

In [8]:
class TransformerEncoder(layers.Layer):
  def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
    super().__init__(**kwargs)
    self.embed_dim = embed_dim
    self.dense_dim = dense_dim
    self.num_heads = num_heads
    self.attention = layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=embed_dim)
    self.dense_proj = keras.Sequential(
        [layers.Dense(dense_dim, activation="relu"),
        layers.Dense(embed_dim),]
 )
    self.layernorm_1 = layers.LayerNormalization()
    self.layernorm_2 = layers.LayerNormalization()

  def call(self, inputs, mask=None):
    if mask is not None:
      mask = mask[:, tf.newaxis, :]
    attention_output = self.attention(
        inputs, inputs, attention_mask=mask)
    proj_input = self.layernorm_1(inputs + attention_output)
    proj_output = self.dense_proj(proj_input)
    return self.layernorm_2(proj_input + proj_output)

  def get_config(self):
    config = super().get_config()
    config.update({
        "embed_dim": self.embed_dim,
        "num_heads": self.num_heads,
        "dense_dim": self.dense_dim,
    })
    return config

In [ ]:
vocab_size = 20000
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
    loss="binary_crossentropy",
    metrics=["accuracy"])
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, None, 256)      │     5,120,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_encoder             │ (None, None, 256)      │       543,776 │
│ (TransformerEncoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 256)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,664,033 (21.61 MB)

 Trainable params: 5,664,033 (21.61 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
callbacks = [
  keras.callbacks.ModelCheckpoint("transformer_encoder.keras",
  save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=20,
callbacks=callbacks)


In [ ]:
model = keras.models.load_model(
 "transformer_encoder.keras",
    custom_objects={"TransformerEncoder": TransformerEncoder})

# when using a cudtom layer always specify it when loading the model

#### Encoder with Positional Embedding

the positional embedding is used to inject order to raw text

it consist of what the embedding layer does but adds  a position vector, which represents the position of the word in the current sentence thefore retaining it's order in the sentence

In [22]:
class PositionalEmbedding(layers.Layer):
  def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
    super().__init__(**kwargs)
    self.token_embeddings = layers.Embedding(
        input_dim=input_dim, output_dim=output_dim)
    self.position_embeddings = layers.Embedding(
        input_dim=sequence_length, output_dim=output_dim)
    self.sequence_length = sequence_length
    self.input_dim = input_dim
    self.output_dim = output_dim

  def call(self, inputs):
    length = tf.shape(inputs)[-1]
    positions = tf.range(start=0, limit=length, delta=1)
    embedded_tokens = self.token_embeddings(inputs)
    embedded_positions = self.position_embeddings(positions)
    return embedded_tokens + embedded_positions

  def get_config(self):
    config = super().get_config()
    config.update({
    "output_dim": self.output_dim,
     "sequence_length": self.sequence_length,
    "input_dim": self.input_dim,
    })
    return config

In [23]:
vocab_size = 20000
sequence_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32
inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs, outputs)

model.compile(optimizer="rmsprop",
    loss="binary_crossentropy",
    metrics=["accuracy"])

model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ positional_embedding_4          │ (None, None, 256)      │     5,273,600 │
│ (PositionalEmbedding)           │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_encoder_1           │ (None, None, 256)      │       543,776 │
│ (TransformerEncoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 256)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,817,633 (22.19 MB)

 Trainable params: 5,817,633 (22.19 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
callbacks = [
  keras.callbacks.ModelCheckpoint("full_transformer_encoder.keras",
  save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=20,
callbacks=callbacks)

model = keras.models.load_model(
"full_transformer_encoder.keras",
  custom_objects={"TransformerEncoder": TransformerEncoder, # when using a custom object it's good to call it when loadin the model
"PositionalEmbedding": PositionalEmbedding})

print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

## Machine Translation

#### Loading the dataset for ML translation

In [12]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Create a directory in Google Drive to store the dataset
!mkdir -p /content/drive/MyDrive/datasets
%cd /content/drive/MyDrive/datasets

# Step 3: Download the dataset using wget
!wget http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip

# Step 4: Unzip the downloaded file
!unzip -q spa-eng.zip


Mounted at /content/drive
/content/drive/MyDrive/datasets
--2025-08-17 22:05:04--  http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.8.207, 142.251.170.207, 173.194.174.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.8.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2638744 (2.5M) [application/zip]
Saving to: ‘spa-eng.zip.2’

spa-eng.zip.2       100%[===================>]   2.52M  2.79MB/s    in 0.9s    

2025-08-17 22:05:06 (2.79 MB/s) - ‘spa-eng.zip.2’ saved [2638744/2638744]

replace spa-eng/_about.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace spa-eng/spa.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


#### preparing the dataset for ML translation

In [13]:
text_file = "/content/drive/MyDrive/datasets/spa-eng/spa.txt"
with open(text_file) as f:
  lines = f.read().split("\n")[:-1]
text_pairs = []

for line in lines:
  english, spanish = line.split("\t")
  spanish = "[start] " + spanish + " [end]"
  text_pairs.append((english, spanish))

In [14]:
import random
print(random.choice(text_pairs))

('I never knew his first name.', '[start] Nunca supe su nombre de pila. [end]')


In [15]:
import random
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples


train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

In [16]:
# Vectorizing the English and Spanish text pairs
import tensorflow as tf
import string
import re
from tensorflow.keras import layers

strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(input_string):
  lowercase = tf.strings.lower(input_string)
  return tf.strings.regex_replace(
    lowercase, f"[{re.escape(strip_chars)}]", "")

vocab_size = 15000
sequence_length = 20

source_vectorization = layers.TextVectorization(
  max_tokens=vocab_size,
  output_mode="int",
  output_sequence_length=sequence_length,
)

target_vectorization = layers.TextVectorization(
  max_tokens=vocab_size,
  output_mode="int",
  output_sequence_length=sequence_length + 1,
  standardize=custom_standardization,
)
train_english_texts = [pair[0] for pair in train_pairs]
train_spanish_texts = [pair[1] for pair in train_pairs]

source_vectorization.adapt(train_english_texts)   # this adapt the vecorization to the train_english_text
target_vectorization.adapt(train_spanish_texts)

In [17]:
batch_size = 64
def format_dataset(eng, spa):
  eng = source_vectorization(eng) # this applies the vectorization to the train ds
  spa = target_vectorization(spa)
  return ({
    "english": eng,
    "spanish": spa[:, :-1],   # the first part ie the dictionary is the inputs while
  }, spa[:, 1:])  # this last part is the traget as rnn tries to predict the next word in the sequence

def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs) # this returns a tuple after spliting the pairs(the pairs were defined above)
    eng_texts = list(eng_texts) # truns the tuple to list
    spa_texts = list(spa_texts)

    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts)) # creates a tf.data.Dataset object with the eng and spa pair as input and target
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4) # applies the format_dataset function which further split the dataset object and set the a
                                                                # last word in the spanish part to be the output

    return dataset.shuffle(2048).prefetch(16).cache() # shuffles, prefetch to aid easier processing, cache to save to memory


train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)



#### Sequence to Sequence ML translation with RNN

In [ ]:
# the encoder using GRU rnn

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

embed_dim = 256
latent_dim = 1024
source = keras.Input(shape=(None,), dtype="int64", name="english")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source)
encoded_source = layers.Bidirectional(
  layers.GRU(latent_dim), merge_mode="sum")(x)

In [ ]:
# GRU based Decoder

past_target = keras.Input(shape=(None,), dtype="int64", name="spanish")
x1 = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target) # Ensures embedding on past targets

decoder_gru = layers.GRU(latent_dim, return_sequences=True)
x = decoder_gru(x1, initial_state=encoded_source) # this ensure it the decoder takes in the past_target input while using the initial state of the source
x = layers.Dropout(0.5)(x)
target_next_step = layers.Dense(vocab_size, activation="softmax")(x)
seq2seq_rnn = keras.Model([source, past_target], target_next_step)



In [ ]:
seq2seq_rnn.compile(
  optimizer="rmsprop",
  loss="sparse_categorical_crossentropy",
  metrics=["accuracy"])
seq2seq_rnn.fit(train_ds, epochs=15, validation_data=val_ds)

In [ ]:
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence]) # uses the source vectorization done above
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
      tokenized_target_sentence = target_vectorization([decoded_sentence]) # applies the target vecorization to the decodeded output
      next_token_predictions = seq2seq_rnn.predict(
        [tokenized_input_sentence, tokenized_target_sentence])     # predict the next word give the source and past_target
      sampled_token_index = np.argmax(next_token_predictions[0, i, :])

      sampled_token = spa_index_lookup[sampled_token_index]   # uses the lookup to get the actual text for each vertorized value
      decoded_sentence += " " + sampled_token      # adds those text back to make it a complete sentence
      if sampled_token == "[end]":
        break

    return decoded_sentence


In [ ]:
test_eng_texts = [pair[0] for pair in test_pairs]

for _ in range(20):
  input_sentence = random.choice(test_eng_texts)
  print("-")
  print(input_sentence)
  print(decode_sequence(input_sentence))

#### Sequence-to-sequence learning with Transformer

The Decoder

In [24]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential([layers.Dense(dense_dim, activation="relu"),
            layers.Dense(embed_dim),])
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
        "embed_dim": self.embed_dim,
        "num_heads": self.num_heads,
        "dense_dim": self.dense_dim,
        })
        return config


    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
            tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        else:
            padding_mask = causal_mask # Ensure padding_mask is defined even if mask is None


        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,)

        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

In [25]:
embed_dim = 256
dense_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = PositionalEmbedding(sequence_length + 1, vocab_size, embed_dim)(decoder_inputs) # Added +1 to sequence_length for target
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)

decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])

In [ ]:
transformer.fit(train_ds, epochs=30, validation_data=val_ds)

Testing The transformer

In [ ]:
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"

    for i in range(max_decoded_sentence_length):
      tokenized_target_sentence = target_vectorization([decoded_sentence])[:, :-1]
      predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])
      sampled_token_index = np.argmax(predictions[0, i, :])
      sampled_token = spa_index_lookup[sampled_token_index]
      decoded_sentence += " " + sampled_token
      if sampled_token == "[end]":
        break
    return decoded_sentence

In [ ]:
test_eng_texts = [pair[0] for pair in test_pairs]

for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))